Привет, меня зовут Артем. Сегодня я проверю твой проект.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу! Проект сделан очень хорошо!
* Выводы очень информативны и были бы полезны банку.
* В следующий раз лучше разделять большие ячейки на несколько маленьких и писать комментарии не в самих ячейках, а в markdown. Так намного легче читать ноутбук.
* В работе нет критических замечаний, но обрати внимание на советы, постарайся использовать их в будущих проектах.
* Поздравляю! Ты успешно прошел ревью с первого раза, а это большая редкость! Продолжай в том же духе!

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

По первому взгляду на таблицу методом info() видно, что есть пропуски в общем трудовом стаже ('days_employed') и ежемесячном доходе ('total_income'). Также общий трудовой стаж имеет тип данных float, хотя количество дней число целое. 
По первым пяти строкам видно, что колонки трудового стажа ('days_employed') и образовании ('education') заполнены некорректно. Название колонок однообразно и не требует коррекции.

<div class="alert alert-block alert-success">
<b>Успех:</b> Первичный анализ данных сделан очень хорошо! Стоит еще посмотреть на <span style="font-family: monospace"> .describe() </span>, возможно удастся заметить некоторые артефакты в данных.
</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#проверим допущены ли пропуски в одних и тех же строках
if list(data[data['total_income'].isna()].index) == list(data[data['total_income'].isna()].index):
    print('Пропуски допущены в одних и тех же строках')
else:
    print('Пропуски допущены в разных строках')

#приведем стаж к положительному числу
data['days_employed'] = data['days_employed'].apply(lambda x: abs(x))

#проверим отношения медианы к среднему, чтобы понять чем заполнить пропуски
print('Отношение среднего к медиане для стажа: ', data['days_employed'].mean() / data['days_employed'].median())
print('Отношение среднего к медиане для дохода: ', data['total_income'].mean() / data['total_income'].median())

#посмотрим на значения медианы и среднего по стажу
print('Медиана стажа: {} | Среднее стажа: {}'.format(data['days_employed'].median(), data['days_employed'].mean()))

#заполним пропуски в стаже медианой
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

#заполним пропуски в доходе средним
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

Пропуски допущены в одних и тех же строках
Отношение среднего к медиане для стажа:  30.495899052668786
Отношение среднего к медиане для дохода:  1.1544937478551978
Медиана стажа: 2194.220566878695 | Среднее стажа: 66914.72890682236


<div class="alert alert-block alert-info">
<b>Совет: </b> Можно было сделать так: <span style="font-family: monospace"> .apply(abs) </span>
</div>

### Вывод

Пропуски допущены в одних строках и скорее всего взаимосвязаны. Если нет официального подтверждения трудового стажа, то и предоставить данные о доходах проблематично. При ознакомлении стало ясно, что все значения стажа отрицательны, что нелогично. Скорее всего данная ошибка появилась из-за того, что были перепутаны даты начала и окончания работы. Для того, чтобы понять чем заполнять пропуски определили отношения средннего к медиане. Для дохода данное отношение близко к еденице, что говорит нам о схожести этих величин. Т.е. для замены мы можем использовать любую. Было выбрано среднее. Для стажа данное отношение сильно отличается от еденицы. При просмотре на значения, видно, что среднее значение слишком большое и не подходит для определения стажа. Такое значение среднего обуславливается артефактами в данных. Поэтому для замены пропусков стада используем медиану.

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что сделал проверку того, что пропуски в одних и тех же строках! Мало кто обращает на это внимание! Предположение по поводу не официального трудоустройства очень интересное и похоже на правду.
    <br> Выбор медианы и среднего аргументирован и абсолютно логичен!
</div>

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Количество отработанных дней должно быть целым числом. Привели данную колонку к типу int. Пропущенные значения были заполнены в предидущем шаге.

<div class="alert alert-block alert-success">
<b>Успех:</b> Согласен, что не целое количество дней не имеет смысла.
</div>

### Обработка дубликатов

In [5]:
#оценим количество дубликатов
print('Количество дубликатов: ', data.duplicated().sum())

#посмотрим на уникальные значения в ячейках с образованием, типом занятости и семенймы положением
print('Уникальные значения в колонке образование: ', data['education'].unique())
print('Уникальные значения в колонке семейное положение: ', data['family_status'].unique())
print('Уникальные значение в каолонке типа занятости: ', data['income_type'].unique())

#приводим столбец с образованием к единообразию
data['education'] = data['education'].str.lower()

#теперь посмотрим изменилось ли количество дубликатов
print('Количество дубликатов после преобразований:', data.duplicated().sum())

#удалим дубликаты и проверим результат
data = data.drop_duplicates().reset_index(drop=True)
print('Проверка:', data.duplicated().sum())

Количество дубликатов:  54
Уникальные значения в колонке образование:  ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальные значения в колонке семейное положение:  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные значение в каолонке типа занятости:  ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Количество дубликатов после преобразований: 71
Проверка: 0


### Вывод

Оценили количество дубликатов, начальное число 54. Но в наших данных есть столбцы которые можно заполнить по разному. При ознакомлении с таблицой очевидны было разное написание образование. Проверили такие столбцы на уникальные значения. После привели все записи образования в единый вид. После этих преобразований количество дубликатов увеличилось до 71, от которых успешно избавились. Возможно дубликаты появились из-за того, что человек приходил в разные отделения для подачи заявки на кредит или попадал к разным менеджерам в одном отделении.

<div class="alert alert-block alert-success">
<b>Успех:</b> Восхитительно!
</div>

### Лемматизация

In [5]:
import pymystem3 as pm
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

#приведем все знаки в цели кредита к нижнему регистру
data['purpose'] = data['purpose'].str.lower()

#создадим словарь возможных целей
dict_purpose = list(map(russian_stemmer.stem, ['жилье', 'свадьба', 'автомобиль', 'образование', 'недвижимость']))

#напишем функцию которая сделает группы на основе строк из исходной таблицы
def define_purpose(purpose):
    
    #лемматизируем принятую строку
    lem_purpose = sorted(pm.Mystem().lemmatize(purpose), reverse = True)
    
    #соотносим цель со словарем
    for word in lem_purpose:
        if russian_stemmer.stem(word) in dict_purpose:
            return word
    #возвращаем исходную строку если не нашлось совпадений
    return purpose


#находим список уникальных целей
unique_purpose_raw = data['purpose'].unique()

#создаем пустой словарь с ключами из списка уникальных целей
purposes_dict_for_replace = dict.fromkeys(unique_purpose_raw)

#заполняем словарь лемматизированными словами
for i in purposes_dict_for_replace:
    purposes_dict_for_replace[i] = define_purpose(str(i))

#создаем новую колонку, которая характеризует цель кредита одним словом
purpose_tag = data['purpose'].apply(lambda x: purposes_dict_for_replace[x])
purpose_tag.name = 'purpose_tag'
try:
    data = data.join(purpose_tag)
except:
    pass

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_tag
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,жилье


### Вывод

С помощью библиотке PyMystem и NLTK обработали цели взятия кредита и получили единообразное описание. Используя новую колонку можно будет категоризировать данные и делать на их основе выводы. Функция работает, к сожалению, долго.

<div class="alert alert-block alert-info">
<b>Совет: </b> В целом все сделано хорошо и работает абсолютно корректно. Но отмечу возможные улучшения:
    <br> - Зачем использовать сортировку в функции <span style="font-family: monospace"> define_purpose </span>?
    <br> - Объект <span style="font-family: monospace"> pm.Mystem() </span> можно было инициализировать один раз, а не при каждом вызове функции.
    <br> - Вместо <span style="font-family: monospace"> .apply() </span> можно было использовать <span style="font-family: monospace"> .map() </span>. Подробнее можешь прочитать <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html"> в документации</a>.
    <br> Часть этих улучшений ускорит (возможно не очень заметно) выполнения ячейки.
</div>

### Категоризация данных

In [6]:
print(data.groupby('education').agg({'debt' : 'count'}).rename(columns={'debt' : 'количество заемщиков'}))
print(data.groupby('family_status').agg({'debt' : 'count'}).rename(columns={'debt' : 'количество заемщиков'}))

                     количество заемщиков
education                                
высшее                               5250
начальное                             282
неоконченное высшее                   744
среднее                             15172
ученая степень                          6
                       количество заемщиков
family_status                              
Не женат / не замужем                  2810
в разводе                              1195
вдовец / вдова                          959
гражданский брак                       4151
женат / замужем                       12339


### Вывод

Посчитали количество заемщиков в разных группах по образованию и семейному статусу. Самыми распространенными группами оказались люди со средним образованием и находящиеся в браке.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#создадим новый dataframe для анализа
children_data = data[['children', 'debt']]

#функция для создания категории "с детьми / без детей"
def child_status(child):
    if child != 0:
        return 'есть дети'
    else:
        return 'нет детей'

#применяем функцию к новому dataframe и добавляем новую колонку со статусом
status = children_data['children'].apply(child_status)
status.name = 'status'
try:
    children_data = children_data.join(status)
except:
    pass

#найдем количество должников в разных группах
debt_w_child = children_data[children_data['debt'] == 1].groupby('status').agg({'debt' : 'count'}) 

#найдем долю должников относительно всех заемщиков
result = debt_w_child['debt'] / children_data.groupby('status')['debt'].count() * 100
print(result)

status
есть дети    9.208203
нет детей    7.543822
Name: debt, dtype: float64


### Вывод

Среди людей с детьми, обратившимися за кредитом 9.2% имели задолженности. Тогда как среди людей без детей таких только 7.5%. Отсюда можно сделать вывод, что наличие детей усложняет возврат кредита вовремя.

<div class="alert alert-block alert-success">
<b>Успех:</b> Хорошо, что была создана новая колонка, я бы поступил точно также.
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
#создадим новые dataframe для анализа
family = data[['family_status', 'debt']]

#найдем количество должников в разных группах
family_w_debt = family[family['debt'] == 1].groupby('family_status').agg({'debt' : 'count'})

#найдем долю должников относительно всех заемщиков
result = family_w_debt / family.groupby('family_status').agg({'debt' : 'count'}) * 100
result = result.reset_index()
result.debt = result.debt.apply(lambda x: float('{:.2f}'.format(x)))
print(result.sort_values(by='debt', ascending=True))

           family_status  debt
2         вдовец / вдова  6.57
1              в разводе  7.11
4        женат / замужем  7.55
3       гражданский брак  9.35
0  Не женат / не замужем  9.75


### Вывод

Результаты говорят, что те люди которые официально оформляли отношения более дисциплинированы в выплатах, в сравнении с тем кто живет в гражданском браке или без официального оформления отношений. 

<div class="alert alert-block alert-success">
<b>Успех:</b> Расчеты и выводы абсолютно корректны!
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
#определяем первый и третий квартиль для дохода
first_quartile = int(data['total_income'].quantile([.25]))
third_quartile = int(data['total_income'].quantile([.75]))

#создаем новый dataframe для анализа
income_data = data[['debt', 'total_income']]

#напишем функцию для ранжирования дохода
def income_range(income, low_level, high_level):
    if income < first_quartile:
        return 'низкий'
    elif income < third_quartile:
        return 'средний'
    else:
        return 'высокий'

#применяем функцию к таблице и вносим новые данные в столбец level
level_series = income_data['total_income'].apply(lambda x: income_range(x, first_quartile, third_quartile))
level_series.name = 'level'
income_data = income_data.join(level_series)

#найдем долю людей с задолженностью внутри разных групп дохода
result = income_data[income_data['debt'] == 1].groupby('level').count()['debt'] / income_data.groupby('level').count()['debt']  
print(result.sort_values(ascending=True))

level
высокий    0.071402
низкий     0.079605
средний    0.086798
Name: debt, dtype: float64


### Вывод

Наибольший процент людей с задолженностью среди людей со средним уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#создаем новый dataframe для анализа
income_data = data[['debt', 'purpose_tag']]

#найдем долю людей с задолженностью внутри разных групп целей
result = income_data[income_data['debt'] == 1].groupby('purpose_tag').count() / income_data.groupby('purpose_tag').count() * 100
print(result.sort_values(by='debt', ascending=True))

                  debt
purpose_tag           
жилье         6.905830
недвижимость  7.463392
свадьба       8.003442
образование   9.220035
автомобиль    9.359034


### Вывод

Люди берущие кредит на операции с недвижимостью имеют самую низкую долю задолженности. А самыми рисковыми оказались образование и автомобиль.

<div class="alert alert-block alert-success">
<b>Успех:</b> В целом шаг сделан очень хорошо. Иногда бывает полезно посмотреть на размеры групп. Например, если кредит для свадьбы брали только 10 человек, а на жилье 10000, то делать выводы не очень корректно.
</div>

### Шаг 4. Общий вывод

Оказалось, что лучшим заемщиком является вдовец или вдова без детей и с высоким уровнем дохода, собирающийся купить жилье! А худшим не состоящий в официальных отношениях человек с ребенком, со средним уровнем дохода и желающим купить автомобиль!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.